In [395]:
import numpy as np
from numpy import *
import pandas as pd
from matplotlib import pyplot as plt
import math
import random
from tqdm import tqdm
from collections import defaultdict
import warnings
from scipy.stats import norm
from sklearn.metrics import confusion_matrix
from collections import Counter

In [2]:
df=pd.read_csv(r"heart.csv")

In [3]:
df.tail()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
298,57,0,0,140,241,0,1,123,1,0.2,1,0,3,0
299,45,1,3,110,264,0,1,132,0,1.2,1,0,3,0
300,68,1,0,144,193,1,1,141,0,3.4,1,2,3,0
301,57,1,0,130,131,0,1,115,1,1.2,1,1,3,0
302,57,0,1,130,236,0,0,174,0,0.0,1,1,2,0


`age
sex
chest pain type (4 values)
resting blood pressure
serum cholestoral in mg/dl
fasting blood sugar > 120 mg/dl
resting electrocardiographic results (values 0,1,2)
maximum heart rate achieved
exercise induced angina
oldpeak = ST depression induced by exercise relative to rest
the slope of the peak exercise ST segment
number of major vessels (0-3) colored by flourosopy
thal: 3 = normal; 6 = fixed defect; 7 = reversable defect`

### Naive Bayes classifier: Split into training and test sets (8:2)
### k-NN classifier: Split into training, validation, and test sets (6:2:2)
`validation set is used to fine-tune the hyper-parameter k`

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trestbps  303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    int64  
 6   restecg   303 non-null    int64  
 7   thalach   303 non-null    int64  
 8   exang     303 non-null    int64  
 9   oldpeak   303 non-null    float64
 10  slope     303 non-null    int64  
 11  ca        303 non-null    int64  
 12  thal      303 non-null    int64  
 13  target    303 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 33.3 KB


In [5]:
df.isnull().sum()

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64

In [6]:
X = df.drop(['target'],axis=1)
y = df['target']

# Naive Bayes classifier

In [7]:
from sklearn.model_selection import train_test_split
X_train , X_test , y_train , y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [8]:
print(X_train.shape,X_test.shape)
print(y_train.shape,y_test.shape)

(242, 13) (61, 13)
(242,) (61,)


In [9]:
X_test

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
179,57,1,0,150,276,0,0,112,1,0.6,1,1,1
228,59,1,3,170,288,0,0,159,0,0.2,1,0,3
111,57,1,2,150,126,1,1,173,0,0.2,2,1,3
246,56,0,0,134,409,0,0,150,1,1.9,1,2,3
60,71,0,2,110,265,1,0,130,0,0.0,2,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
249,69,1,2,140,254,0,0,146,0,2.0,1,3,3
104,50,1,2,129,196,0,1,163,0,0.0,2,0,2
300,68,1,0,144,193,1,1,141,0,3.4,1,2,3
193,60,1,0,145,282,0,0,142,1,2.8,1,2,3


In [233]:
# calculate probarility of P(y=j)
def estimate_target_prob(df, target, label):
    return len(df[target==label])/len(df)

In [226]:
# calculate probarility of P(y=j) against categorical variable 
def category_prob(df, column, category, target, label, l=1):
    numerator = len(df[(df[column]==category)&(target==label)]) + l
    denominator = len(df[target==label])+l*len(df[column].unique())
    return numerator/denominator

In [220]:
# calculate probarility against continuous variable 
def estimate_mean_std(df, column, target, label):
    temp = df[target==label]
    mu = temp[column].mean()
    std = temp[column].std()
    return mu, std

In [224]:
def train(X,y):
    categorical_column = ['sex', 'cp', 'fbs','restecg','exang','slope','ca','thal']
    unique_label = y.unique()
    reference_dict = dict()
    for col in X: 
        temp_res = []
        for label in unique_label:
            if col in categorical_column:
                unique_value = X[col].unique()
                temp_res1 = []
                for u in unique_value:
                    temp_res1.append(category_prob(X, col, u, y, label, l=1))
                temp_res.append(temp_res1)
            else:
                temp_res.append(estimate_mean_std(X, col, y, label))

        reference_dict[col] = temp_res

    return reference_dict

In [231]:
def prior(X,y):
    unique_label = y.unique()
    prior_probs = []
    for label in unique_label:
        prior_prob = estimate_target_prob(X, y, label)
        prior_probs.append(prior_prob)
    return prior_probs

In [234]:
reference_dict = train(X_train,y_train)
prior_probs = prior(X_train,y_train)

In [235]:
def predict(X, y, ref, prior):
    categorical_column = ['sex', 'cp', 'fbs','restecg','exang','slope','ca','thal']
    unique_label = y.unique()
    object_value = [0]*len(unique_label)
            
    for idx in new_data.index[:-1]:
        value = new_data[idx] 
        reference_value = ref[idx]

        if idx in categorical_column:
            for i, r in enumerate(reference_value):
                ## if category, reference value is the value of probability
                value = int(value) 
                object_value[i] += np.log(r[value])
        else:
            for i, r in enumerate(reference_value):
                ## if continuous, reference value contains (mean, std)
                object_value[i] += np.log(norm.pdf(value, r[0], r[1]))

    for i in unique_label:
        object_value[i] += np.log(prior[i])
    max_object_value = max(object_value)
    max_idx = object_value.index(max_object_value)
    return max_idx

In [302]:
# test model
y_pred=[]
for i in range(len(X_test)):
    y_pred.append(predict(X_test.iloc[i],y_train,reference_dict,prior_probs))

In [326]:
# calculate accuracy
true_positive = 0
true_negative = 0
false_positive = 0
false_negative = 0

for i in range(len(y_pred)):
    if y_pred[i]==1 and y_test.iloc[i]==1:
        true_positive +=1
    if y_pred[i]==0 and y_test.iloc[i]==1:
        true_negative +=1
    if y_pred[i]==1 and y_test.iloc[i]==0:
        false_positive +=1
    if y_pred[i]==0 and y_test.iloc[i]==0:
        false_negative +=1
print(true_positive, true_negative, false_positive, false_negative)

32 0 29 0


In [330]:
accuracy = (true_positive+true_negative)/len(y_test)
recall = (true_positive)/(true_positive+false_negative)
precision = true_positive/(true_positive+false_positive)
f1score = 2*(precision*recall)/(precision+recall)

In [331]:
print(accuracy, recall, precision, f1score)

0.5245901639344263 1.0 0.5245901639344263 0.6881720430107526


In [306]:
confusion_matrix(y_test, y_pred)

array([[ 0, 29],
       [ 0, 32]], dtype=int64)

# KNN classifier

In [345]:
# Split data into train, valid and test subsets(6:2:2)
X_train , X_test , y_train , y_test = train_test_split(X,y,test_size=0.2,random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

In [383]:
class MyScaler:
    def fit(self, X):
        """compute mean and stdev of each columns"""
        # mean
        self.feature_means = np.mean(X, axis=0) 
        # stdev
        self.feature_stds = np.std(X, axis=0)

    def transform(self, X):
        
       
        dim = X.shape
        transformed = np.empty(dim)
        for row in range(dim[0]): 
            for col in range(dim[1]): 
                # x_new = (x - mean) / std
                transformed[row, col] = (X[X.columns[col]].iloc[row] - self.feature_means[col]) / \
                self.feature_stds[col]
        return transformed

In [384]:
scaler = MyScaler() 
scaler.fit(X_train) 

In [385]:
X_train_scaled = scaler.transform(X_train)
print(X_train_scaled)
X_test_scaled = scaler.transform(X_test)
print(X_test_scaled)

[[-1.4307838   0.69541826  0.08801311 ...  0.93223831 -0.65423295
  -0.50215723]
 [ 0.27438076  0.69541826  0.08801311 ...  0.93223831 -0.65423295
   1.18099941]
 [ 1.76639975 -1.43798352 -0.90763525 ... -0.67476297 -0.65423295
  -0.50215723]
 ...
 [-0.25848316  0.69541826  2.07930984 ... -0.67476297 -0.65423295
   1.18099941]
 [ 0.38095355 -1.43798352  2.07930984 ...  0.93223831 -0.65423295
  -0.50215723]
 [ 1.34010861 -1.43798352  1.08366148 ... -0.67476297 -0.65423295
   1.18099941]]
[[ 0.27438076  0.69541826 -0.90763525  1.29651504  0.4702916  -0.39098113
  -1.01220856 -1.7172935   1.45625879 -0.39131854 -0.67476297  0.43215387
  -2.18531386]
 [ 0.48752633  0.69541826  2.07930984  2.55987559  0.6881362  -0.39098113
  -1.01220856  0.43109118 -0.68669114 -0.74768914 -0.67476297 -0.65423295
   1.18099941]
 [ 0.27438076  0.69541826  1.08366148  1.29651504 -2.25276595  2.55766821
   0.83839497  1.07103555 -0.68669114 -0.74768914  0.93223831  0.43215387
   1.18099941]
 [ 0.16780798 -1.43

   1.18099941]]


In [402]:
class MyKnnClassifier:
    def __init__(self, n_neighbors=5): 

        self.k = n_neighbors

    def fit(self, X_train, y_label): 

        self.points = X_train
        self.labels = list(y_label)
        
    def majority_vote(self, distances):
        indices_by_distance = np.argsort(distances)
        print('indices_by_distance:', indices_by_distance)
        k_nearest_neighbor = []
        for i in range(self.k):
            idx = indices_by_distance[i]
            k_nearest_neighbor.append(self.labels[idx])

        print('k_nearest_neighbor: ',k_nearest_neighbor)

        vote_counts = Counter(k_nearest_neighbor)
        print('vote_counts: ', vote_counts)


        print('vote_counts.most_common(2): ', vote_counts.most_common(2)) 
        winner, winner_count = vote_counts.most_common(1)[0]
        return winner
        
    def predict(self, X_test): 

        predicts = [] 
        for test_pt in X_test: 
            distances = self.distance(self.points, test_pt)
            print(test_pt)
            print(distances)
     
            winner = self.majority_vote(distances)
            
            predicts.append(winner)

        return np.array(predicts)
    
    def distance(self, X, y):
      
        print('print distance:', np.sqrt(np.sum((X - y) ** 2, axis=1)))
        return np.sqrt(np.sum((X - y) ** 2, axis=1))
    


In [404]:
knn = MyKnnClassifier(n_neighbors=3) 
print('k =', knn.k)
knn.fit(X_train_scaled, y_train)
y_pred=knn.predict(X_test_scaled)

k = 3
print distance: [5.40813898 5.35753803 5.57062731 5.05941714 6.28514011 4.25959534
 5.21762069 5.37281464 5.38544499 5.04195144 6.09387928 3.13018874
 6.13260904 5.8045343  4.49491629 4.48840198 4.65101254 5.65568628
 5.69213158 5.15005113 4.71073213 4.6942452  4.69800906 4.95488673
 5.05990749 6.44968204 5.28438338 4.58258817 5.35555243 6.66771534
 5.82664349 5.30914382 5.17918241 5.46153892 5.9052813  6.21610217
 4.4496218  6.2019557  5.04233138 6.58546858 6.10684762 3.24894764
 5.09370011 5.90699975 4.08636246 5.63963295 5.57700961 4.83827142
 5.23249141 5.26691852 3.15276316 5.41711577 6.23156343 7.08095959
 5.39628037 5.59372369 5.76242598 5.18807103 4.10695963 4.03278324
 4.92304942 6.7850545  5.38384359 5.37291842 5.00422936 5.90717809
 4.02683754 3.52514904 4.09933643 5.20370058 5.55900466 5.29813665
 6.13848226 5.44225338 5.56599656 5.42971964 5.41027139 5.22455008
 4.03479188 3.80544005 5.02746217 5.91189831 5.74740072 5.16834537
 4.4310156  5.23411257 5.12387552 6.7210

 6.70836616]
indices_by_distance: [ 94   3 156  91 140 138 121   8  37 178 150 165 113  83 129 168  60  30
 120 102  69 160 110 142  65  87  32 114   1 151 177  47 170 148  45 179
 103 115  49  86  14  54 109  31 157  21  35 125 158 128  77 163  28  95
  13 152  17 108  22 131 159 134  76   7 167 107 137   9  58 116  88 127
 105  43 176  74  56  71  20  48  93  23 155  16 106 161 136 139 111  68
  26  97  10  19  84 130 172  18   6  96 175 135  27  55  59 104   0 117
  62  66  73  34  57  25 169 101  39 154  51 100  40  36 122 118  80  90
  78  11  64 126   5  38  75  44  12 124  99  46 171  79  41 145 141  81
  85  24  52  98 133 149  33  63  50  42 147  89 146 153  53  67 112  15
 166 144 173 180   2   4  70 174  72 123 119 143  82 162 132  61 164  92
  29]
k_nearest_neighbor:  [1, 0, 1]
vote_counts:  Counter({1: 2, 0: 1})
vote_counts.most_common(2):  [(1, 2), (0, 1)]
print distance: [5.27010454 4.66016746 6.19185046 4.61154105 5.05659715 6.65956107
 5.52514625 4.65051282 4.37186518 

 4.92860949]
indices_by_distance: [ 14 153 111 134  76  36  63  84 160  66 143  24 113  16 166 124 110 169
 144  85 146 174  41  93  71 163 152  97   9  11 107  68 133 102  57   5
  42  32  89 158  47 138 180  59 101 175 135 168 157  44 115 104   3 127
  86  28  67  33 156 131 147 142 125  88   6 167 114  58 100  69 148 128
  73 123 132 105 171  79  50  20   1 151 120 149 129  77  70 165  72 145
  83 139 121  94   8  27  74  19 103 126 176  99 177  43  98  55  48 150
 170  91 172 137  80  31 122 159  51  21 161  81  82 141 140  15  64   7
  23  56 108 109  26  35 173  30  65  60  75  78  49  37  22 179  54  90
  61  96   0 155 164 162  38 136  52 178 118 130 117  46   2  17  39  10
 116  29  45  92  34  95  13 154  62  25  12 112  53  40  87 106  18   4
 119]
k_nearest_neighbor:  [0, 0, 0]
vote_counts:  Counter({0: 3})
vote_counts.most_common(2):  [(0, 3)]
print distance: [5.51961191 5.00129676 5.13633527 4.85491816 3.35977423 3.87112254
 5.16126587 5.23411385 5.03145061 7.92197645 5.7

 8.45605019]
indices_by_distance: [ 21   7 103  95  94 159 177  17  60 155  40  49  13  91  83 170  54  34
   8 140 125  74 121 120  10   1  62 136  69 165 172  31 129 151  37  51
  96   0  26 148  48 142  52  56  80  77  38 102  43 139  87 109  45 128
  19 117   3 115 130   6 114 131 106  86 107 150  27  71 156  47 113  90
 110 108  68 137  25  75  73  88 105  46 168  55  64 167  22  23 178  32
  35  20  99  39 118  65 152  12 179 122 138 161 160  16 157 163 104 171
   2  58  93  28 176  97  15 162 141 112 116  79  76  53 100  70  33 158
  84  85  98 169 127  24  42  50 175 123  18 145 173  92  78  14 126  72
 154  44 143 119 111   4  30  36   5 134 124  66 101 149  41  63  59 147
  81  57  89  61 146   9  11  82 135 133 144  67 164 132 174 166  29 153
 180]
k_nearest_neighbor:  [1, 1, 1]
vote_counts:  Counter({1: 3})
vote_counts.most_common(2):  [(1, 3)]
print distance: [4.42465607 4.6435503  4.92748494 4.3698298  5.89262274 4.14895023
 3.38121512 4.52850362 4.16580104 5.26490721 4.8

 9.2713793 ]
indices_by_distance: [  9  30  81  76 166 153  11 132  82  14  57  36  58  68 169  59  22 164
  44 174 163  84  33  20 157  66 160 113 124 176 111  85 158  49  97 178
 102 101  87 146 134  67 122  65 133 116 179  18 110  60  13 123 126  78
 138 135 127 152  72 145 167  71  63  94  42 173 130 128  31  83  47  69
  50   3  27 143  86 159 140 115  43 170  41  32  19 171 120  98 131  45
 168  16  39 107  74 109   5 142 154 156 172 165  92  21 125  55 137  93
 139 144  37  24 149 129  52 175 151   6   8  89  12  91  77 161   2  23
  28  61 108   1  56 103 121 148  73 147  79 177  29 114  51   4 162 150
  96  35 104  46  54  64   7  70 100  26  88  80  95  48 112  15 155 105
  53 117  38  75 180 118  90  99  17  34  62   0  25  40 136 141 119  10
 106]
k_nearest_neighbor:  [0, 0, 0]
vote_counts:  Counter({0: 3})
vote_counts.most_common(2):  [(0, 3)]
print distance: [5.53636561 5.23929243 5.41825028 4.74612821 6.76491953 4.59953264
 5.76627133 5.459006   5.49151214 4.62345475 6.2

...

 7.48008185]
indices_by_distance: [160 111  36 169  66  41  11  84 175 146 157 124   3  32 144 104  89 174
  14  59 158  19 149  57  76  97 167  71  44  63  42  16 163  79 170 133
 139 156  47 165 114  69  98   8  28  85  20 143  21  80 152 145  75  72
  33  55 110  91 151   1 125 126 137  99  61 113 112 128 141  67 101  94
  48 177 150 103 105   9  86 122  73  26  68 102 131  54 115 118 159 148
  90  58 153  24   5  50 134  93  15 173 100 161 172   6 155 171 140 108
   0 142 147  31  38  78  64 166 127 176 123  70 121   7 109  17 107  27
  96  77  34 132  35  81 136  95 120 168  25  10  22  37  56 129  23 138
  60  51  74  40  30 162  65  49  88  46  83  43  45  52 117 116  62 106
 178 135  53 130  92 154  87  39  13 179 164   2  82  29 180  18   4  12
 119]
k_nearest_neighbor:  [0, 0, 0]
vote_counts:  Counter({0: 3})
vote_counts.most_common(2):  [(0, 3)]
print distance: [3.02791195 4.31905444 4.87892458 4.5889229  5.25019475 5.84385097
 3.90235015 3.76256395 4.70204462 6.76653961 3.7

 5.76427022]
indices_by_distance: [ 88 107  77  70 136 105  16 142   6 148 129 115  51 177 120  74 168   7
 121  24  48  93 117  80  56  10 110 103  47  17   0   1  43 172  99 125
  83 155 100  21 104 114 113 102 141 143   5 152 131  62  38  86  46 109
  91  27 156  73  15  64  35 127  95 147 101   3   8 138  90 134 151 165
 171  75 108  26 150  54  34  31  28  96  40 135  69  79  32 167   2 170
 144 137  23 179  68 139  37 140 161  53 106 111  12  97 160 159  52  94
   4  71  55  41  89  63 119 128  14 118  45 126  65  19  61 162  13 149
 123  76  42 175  60  72 163  85  25  78  84  49 112  50 180  44 178 133
 132 146  66  20  22 122 124  57  58  36 158 176 169  98 154 130 116  39
 157 174  59 173 153  87  67 145 166  33  11  92  18  29  82  30  81   9
 164]
k_nearest_neighbor:  [1, 1, 1]
vote_counts:  Counter({1: 3})
vote_counts.most_common(2):  [(1, 3)]
print distance: [4.26843697 3.40668844 5.88844928 3.23804887 6.70770239 4.76734551
 4.12313715 3.75172576 2.83688891 6.25627525 4.4

 9.77389402]
indices_by_distance: [ 76  33 153 163  57  36 169 164 124 128  84  20  60  85 167   9 122 160
 133  31  49  81  11  66 102  42 132  67 113  68 176 145 123 143  43 127
 146  14   2  63  39 173   3 157  29  27  47  37 110  74  94 130 152  59
  69  86 166  82 172  92  21  55 170 134 116  44 111 161  32  97 174 159
 126  58  30  98  83 156  70  52  18  41  65 131 144  16 158 115 148  23
 171  12 125  72  19 165  71 142 120 137  96   8  13 140 108 109 107 147
  56  46 138 149  51 129   5 101  15  73  22  28 162   1 168   7  35 112
  79  89 151   6  50 103  93 178  80 179  99  75 139  64  77  61  87  91
 177 104 118  26  54  78 154  62  38  95  90  45 114 155  24 135 117  40
 121 175  17 105  34  88   0  48 150 136   4 100 141  25  10 106  53 119
 180]
k_nearest_neighbor:  [0, 0, 0]
vote_counts:  Counter({0: 3})
vote_counts.most_common(2):  [(0, 3)]
print distance: [6.02873621 4.75814661 5.76646344 3.03355495 6.74732759 5.10912787
 6.06416651 5.23053937 4.54445938 5.46154081 6.0

 8.00854975]
indices_by_distance: [ 17  95 155 136 177  21 103   7  40  10  34   0  91  62  48  38  54  80
   1 117   8  77  56 106 165  90 114  26 159 121 109 125  74 142 105 115
   6  51  94 170  75  96  69 150 151 129 156  46   3 148 139  83  88  25
  99  64 108  35 118 131  86 172 104  45  37 120  73 140  13  55  52 141
 107  47  19  27 102 137  60  32  49  53  31 110 168 100 161 152 113  16
  28  71  43  23  87 171  93  79  70 175  15 128  97 162  98 112 167  68
 157  22 178  24  20 130   2 179 160 147 149 116  65 138  39   4  12 119
  41 127  50 158 176 122 154  78  58  42 163 111   5 145 126  44  84  72
 143 101 123  61  85 173  92  14 169  66 134  63  76  59 144  36 124 146
  18  89  33 135  57  67  30 133  81  11 132 174 166   9  82 164  29 153
 180]
k_nearest_neighbor:  [1, 1, 1]
vote_counts:  Counter({1: 3})
vote_counts.most_common(2):  [(1, 3)]
print distance: [6.5400847  5.39550062 5.36029383 5.23456262 5.92953065 6.51340591
 6.35256956 5.70891151 5.76862075 5.78479762 6.9

 7.07143956]
indices_by_distance: [ 36  84 169 124  66  33 163  85 160  11 111  76  63   9 146  42 157  14
 158  59 133  20  67  41  71 144 128 145 174 167  57   3  44 153  19  68
  32 122  89 143  69  79  81  94  60 173  28 149 113 164 170  72 139 156
 102  31  97  98 137 148 110   8 175  21 152 166   5  49   1 134 123  16
  55  23 140  47  30 165 151 125 104  37 159 127  50 126  58 130 171  96
 176  99  22 177 103  64  24  80 114  27  91 131 161  73  86   7  15 162
  93 132 105 107  43 142 138  26 168 141 115 121  75 108  78  65  83  74
  70 172  35  54 101 118  92 120  90 129   6  38  48 147  82  61  51   2
 116 155 150  39  13  77 100   0 178  62  17 136  95  45 109  56  52  88
 135  87  46  34  29 112  10 179  40  18  25  12 154  53 117 106 180   4
 119]
k_nearest_neighbor:  [0, 0, 0]
vote_counts:  Counter({0: 3})
vote_counts.most_common(2):  [(0, 3)]
print distance: [4.88144995 3.86150692 5.03033768 3.54272699 6.29889406 5.63569611
 4.81114638 4.65220995 3.69642244 3.57350602 5.5

In [405]:
# calculate accuracy
true_positive = 0
true_negative = 0
false_positive = 0
false_negative = 0

for i in range(len(y_pred)):
    if y_pred[i]==1 and y_test.iloc[i]==1:
        true_positive +=1
    if y_pred[i]==0 and y_test.iloc[i]==1:
        true_negative +=1
    if y_pred[i]==1 and y_test.iloc[i]==0:
        false_positive +=1
    if y_pred[i]==0 and y_test.iloc[i]==0:
        false_negative +=1
print(true_positive, true_negative, false_positive, false_negative)

29 3 4 25


In [407]:
accuracy = (true_positive+true_negative)/len(y_test)
recall = (true_positive)/(true_positive+false_negative)
precision = true_positive/(true_positive+false_positive)
f1score = 2*(precision*recall)/(precision+recall)
print(accuracy, recall, precision, f1score)

0.5245901639344263 0.5370370370370371 0.8787878787878788 0.6666666666666667


In [408]:
confusion_matrix(y_test, y_pred)

array([[25,  4],
       [ 3, 29]], dtype=int64)

# Comparing Naive Bayes(NB) and KNN
### KNN shows much better performance than NB for the following reasons.
1. NB can suffer from the zero probability problem. When a particular attribute's conditional probability eaquals zero, NB completely fail to produce a valid prediction
2. NB will only work if the decision boundary is linear, elliptic, or parabolic. 